In [1]:
cd ..

/home/tommy/Scrivania/Polimi/Honours Programme/SCFR


In [2]:
from os import listdir, path
import json
import matplotlib.pyplot as plt

from data_structures.cfr_trees import CFRTree
from games.goofspiel import build_goofspiel_tree
from games.kuhn import build_kuhn_tree
from games.leduc import build_leduc_tree
from functools import reduce
import math

In [3]:
def parameters_to_string(par):
    if(par['game'] == 'kuhn'):
        return 'kuhn_' + str(par['players']) + '_' + str(par['rank'])
    if(par['game'] == 'leduc'):
        return 'leduc_' + str(par['players']) + '_' + str(par['suits']) + '_' + str(par['rank'])
    if(par['game'] == 'goofspiel'):
        return 'goofspiel_' + str(par['players']) + '_' + str(par['rank']) + '_' + str(par['tie_solver'])
    if(par['game'] == 'random'):
        return 'random_' + str(par['players']) + '_' + str(par['depth']) + '_' + str(par['branching_factor'])
    if(par['game'] == 'hanabi'):
        return 'hanabi_' + str(par['players']) + '_' + str(par['suits']) + '_' + \
                str(par['color_distribution']).replace(' ','').replace(',','_') + '_' + \
                str(par['cards_per_player']) + '_' + str(par['starting_clue_tokens'])

In [4]:
# TODO: check it, in particular for bigger versions of goofspiel
# TODO: implement different goofspiel TieSolver

def delta_utility(game, players, rank, betting_parameters = [2, 4], color_distribution = None, suits = 1):
    if game == 'kuhn':
        return 2 * players
    if game == 'leduc':
        return players + players * betting_parameters[0] + players * betting_parameters[1]
    if game == 'goofspiel':
        return sum(list(range(1, rank+1)))
    if game == 'random':
        return 1
    if game == 'hanabi':
        return (len(color_distribution) + 1) * suits

In [5]:
def max_social_welfare(game, players, rank, betting_parameters = [2, 4], color_distribution = None, suits = 1):
    if game == 'kuhn' or game == 'leduc':
        return 0
    if game == 'goofspiel':
        return sum(list(range(1, rank+1)))
    if game == 'random':
        return players
    if game == 'hanabi':
        return (len(color_distribution)) * suits * players

In [18]:
target_directory = "results/remote/nips/SCFR/goofspiel/"

In [26]:
file_names = listdir(target_directory)

In [27]:
def analyze(name, target_epsilons, data_per_instance, print_graphs = False):
    if path.isdir(target_directory + name):
        return
    
    print("\n---------------------")
    print("File = " + name)
    
    file = open(target_directory + name, "r")
    data = json.load(file)
    
    print("Game = " + parameters_to_string(data['parameters']))
    
    print("Total iterations = " + str(data['parameters']['number_iterations']))
    print("Check every iteration = " + str(data['parameters']['check_every_iteration']))
    
    # --------- SKIP 2 PLAYERS LEDUC AND GOOFSPIEL (AND RANDOM) ---------
    #if data['parameters']['game'] in ['goofspiel', 'leduc', 'random'] and data['parameters']['players'] == 2:
    #    print("---------------------------")
    #    print("Skipping 2 player versions")
    #    print("---------------------------")
    #    print("")
    #    return
    # ------------------------------------------------------
    
    durations = list(map(lambda el: el['duration'], data['data']))
    
    target_datapoint_amount = int(data['parameters']['number_iterations']) / int(data['parameters']['check_every_iteration'])
    run_time = sum(durations)
    avg_time = run_time / len(durations)
    print("Logged " + str(len(data['data'])) + " out of " + str(target_datapoint_amount) + " datapoints in " + \
          str(run_time) + " seconds (aka " + str(run_time/3600) + " hours)")
    print("Average duration of batch = " + str(avg_time) + " seconds (aka " + str(avg_time/3600) + " hours)")

    if 'total_duration' in data:
        running_time = data['total_duration']
        print("Solved in " + str(running_time) + " seconds")
        print("Solved in " + str(running_time / 60) + " minutes")
        print("Solved in " + str(running_time / 3600) + " hours")
        
    if 'utility' in data:
        u = data['utility']
    elif 'utility' in data['data'][-1]:
        u = data['data'][-1]['utility']
    else:
        u = [-1]
        
    if 'color_distribution' not in data['parameters']:
        data['parameters']['color_distribution'] = []
        
    du = delta_utility(data['parameters']['game'], data['parameters']['players'], data['parameters']['rank'],
                   data['parameters']['betting_parameters'], data['parameters']['color_distribution'], 
                   data['parameters']['suits'])
        
    sw = sum(u)
    print("Final utility = " + str(u))
    print("Final epsilon = " + str(-min(data['data'][-1]['epsilon']) / du) + "%")
    print("Social welfare = " + str(sw))
    max_sw = max_social_welfare(data['parameters']['game'], data['parameters']['players'], 
                                data['parameters']['rank'], data['parameters']['betting_parameters'],
                                data['parameters']['color_distribution'], data['parameters']['suits'])
    if max_sw != 0:
        print("Max social welfare = " + str(max_sw) + " (obtained " + \
              str(sw / max_sw * 100) + "%)")

    if 'total_duration' in data:
        print("Average iterations per second = " + str(data['average_iterations_per_second']))
    else:
        print("Average iterations per second = " + str(data['parameters']['check_every_iteration'] / avg_time))

    iteration_counts = list(map(lambda el: el['iteration_number'], data['data']))
    epsilons = list(map(lambda el: max(0, -min(el['epsilon'])), data['data']))
    percent_epsilons = list(map(lambda el: el / du, epsilons))
    if 'utility' in data['data'][0]:
        utilities = list(map(lambda el: sum(el['utility']), data['data']))
    if max_sw != 0 and 'utility' in data['data'][0]:
        percent_utilities = list(map(lambda el: el / max_sw, utilities))
    #absolute_joint_sizes = list(map(lambda el: el['absolute_joint_size'], data['data']))
    #relative_joint_sizes = list(map(lambda el: el['relative_joint_size'], data['data']))
    #max_plan_frequencies = list(map(lambda el: el['max_plan_frequency'], data['data']))
    #joint_support_sizes = list(map(lambda el: el['joint_support_size'], data['data']))
    
    if print_graphs:
        plt.plot(iteration_counts, epsilons)
        plt.title('Epsilons')
        plt.show()
        plt.plot(iteration_counts, percent_epsilons)
        plt.title('Percent epsilons')
        plt.show()
        if max_sw != 0 and 'utility' in data['data'][0]:
            plt.plot(iteration_counts, percent_utilities)
            plt.title('Percent social welfare')
            plt.show()
    
    time_to_target = []
    iterations_to_target = []
    
    for t in target_epsilons:
        indices_below_t = [i for i in range(len(epsilons)) if percent_epsilons[i] <= t]
        if len(indices_below_t) > 0:
            time_to_t = sum(durations[:indices_below_t[0]+1])
            iterations_to_target.append((indices_below_t[0]+1) * int(data['parameters']['check_every_iteration']))
        else:
            time_to_t = -1
            iterations_to_target.append(-1)
        time_to_target.append(round(time_to_t, 5))
    
    s = parameters_to_string(data['parameters'])
    
    d = {'time_to_target': time_to_target,
        'iterations_to_target': iterations_to_target,
        'parameters': data['parameters'],
        'percent_sw': sw / max_sw * 100 if max_sw > 0 else 0}
    
    if (s + '_average') not in data_per_instance:
        data_per_instance[s + '_average'] = []
    data_per_instance[s + '_average'].append(d)
    
    while s in data_per_instance:
        s += '_'
    data_per_instance[s] = d

In [28]:
target_epsilons = [0.1, 0.05, 0.01, 0.005, 0.001, 0.0005]
data_per_instance = {}

for name in file_names:
    analyze(name, target_epsilons, data_per_instance)
    
# Average if needed
new_data_per_instance = {}
for (s, data) in data_per_instance.items():
    if type(data) != list:
        new_data_per_instance[s] = data
    elif len(data) > 1:
        tot_time = [0 for _ in target_epsilons]
        tot_it = [0 for _ in target_epsilons]
        count = [0 for _ in target_epsilons]
        if 'nodes_amount' in data[0]['parameters']:
            node_count = sum(map(lambda d: d['parameters']['nodes_amount'] + d['parameters']['leaves_amount'], data)) / len(data)
            infoset_count = sum(map(lambda d: d['parameters']['infoset_amount'], data)) / len(data)
        else:
            node_count = -1
            infoset_count = -1
        avg_sw = sum(map(lambda d: d['percent_sw'], data)) / len(data)
        for d in data:
            for i in range(len(target_epsilons)):
                if d['time_to_target'][i] > 0:
                    tot_time[i] += d['time_to_target'][i]
                    tot_it[i] += d['iterations_to_target'][i]
                    count[i] += 1   
        for i in range(len(target_epsilons)):
            if count[i] > 0:
                tot_time[i] /= count[i]
                tot_it[i] /= count[i]
            else:
                tot_time[i] = -1
                tot_it[i] = -1
        # Compute variance
        variance = [-1 for _ in target_epsilons]
        confidence = [-1 for _ in target_epsilons]
        for i in range(len(target_epsilons)):
            if count[i] > 1:
                v = 0
                for d in data:
                    if d['time_to_target'][i] > 0:
                        v += (d['time_to_target'][i] - tot_time[i]) ** 2
                variance[i] = v / (count[i] - 1)
                confidence[i] = 1.96 * math.sqrt(variance[i]) / math.sqrt(count[i])
        new_data_per_instance[s] = {'time_to_target': tot_time,
                                    'time_to_target_variance': variance,
                                    'time_to_target_confidence': confidence,
                                    'iterations_to_target': tot_it,
                                    'parameters': data[0]['parameters'],
                                    'avg_node_count': node_count,
                                    'avg_infoset_count': infoset_count,
                                    'avg_sw': avg_sw}
    
data_per_instance = new_data_per_instance


---------------------
File = 1558442902_2_4_Accumulate
Game = goofspiel_2_4_accumulate
Total iterations = 50000
Check every iteration = 50
Logged 399 out of 1000.0 datapoints in 28279.434049129486 seconds (aka 7.8553983469804125 hours)
Average duration of batch = 70.87577455922177 seconds (aka 0.019687715155339378 hours)
Final utility = [4.553803258145722, 5.184793233082919]
Final epsilon = 0.01667000835419623%
Social welfare = 9.738596491228641
Max social welfare = 10 (obtained 97.38596491228641%)
Average iterations per second = 0.7054596625003573

---------------------
File = 1557828159_3_4_Accumulate
Game = goofspiel_3_4_accumulate
Total iterations = 50000
Check every iteration = 50
Logged 322 out of 1000.0 datapoints in 73647.70874762535 seconds (aka 20.457696874340375 hours)
Average duration of batch = 228.71959238392967 seconds (aka 0.06353322010664712 hours)
Final utility = [3.7857660455486393, 3.148571428571472, 3.023618012422415]
Final epsilon = 0.020815217391299167%
Social w


---------------------
File = 1557828233_3_4_Accumulate
Game = goofspiel_3_4_accumulate
Total iterations = 50000
Check every iteration = 50
Logged 318 out of 1000.0 datapoints in 72534.16610050201 seconds (aka 20.148379472361672 hours)
Average duration of batch = 228.0948619512642 seconds (aka 0.06335968387535117 hours)
Final utility = [3.3255083857441954, 3.502434486373028, 3.1257311320755226]
Final epsilon = 0.022778563941312502%
Social welfare = 9.953674004192745
Max social welfare = 10 (obtained 99.53674004192746%)
Average iterations per second = 0.21920704207130817

---------------------
File = 1557992927_3_4_DiscardIfHigh_
Game = goofspiel_3_4_discard_if_high
Total iterations = 50000
Check every iteration = 50
Logged 395 out of 1000.0 datapoints in 84364.59679913521 seconds (aka 23.434610221982002 hours)
Average duration of batch = 213.58125771932964 seconds (aka 0.05932812714425823 hours)
Final utility = [2.9075042194093506, 2.91474261603379, 3.041666666666743]
Final epsilon = 0

In [29]:
size_data = {
    'kuhn_3_6': (72, 147),
    'kuhn_3_7': (84, 171),
    'kuhn_3_8': (96, 195),
    'kuhn_3_9': (108, 219),
    'kuhn_3_10': (120, 243),
    
    'leduc_3_3_3': (684, 1371),
    'leduc_3_3_4': (1200, 2403),
    'leduc_3_3_5': (1860, 3723),
    'leduc_3_3_6': (2664, 5331),
    'leduc_3_3_7': (3612, 7227),
    'leduc_3_3_8': (4704, 9411),
    
    'goofspiel_3_3_accumulate': (194, 396),
    'goofspiel_3_3_discard_if_all': (194, 396),
    'goofspiel_3_3_discard_if_high': (194, 396),
    'goofspiel_3_3_discard_always': (194, 396),
    'goofspiel_3_4_accumulate': (48966, 98523),
    'goofspiel_3_4_discard_if_all': (48966, 98523),
    'goofspiel_3_4_discard_if_high': (48966, 98523),
    'goofspiel_3_4_discard_always': (48966, 98523),
    
    # Random sized are taken from specific dat files
    'random_2_5_2': (24, 46),
    'random_2_6_2': (52, 102),
    'random_2_7_2': (94, 186),
    'random_2_8_2': (186, 370),
    'random_2_9_2': (383, 764),
    'random_2_10_2': (776, 1550),
    'random_3_5_2': (29, 55),
    'random_3_6_2': (49, 95),
    'random_3_7_2': (100, 197),
    'random_3_8_2': (206, 409),
    'random_3_9_2': (378, 753),
    'random_3_10_2': (757, 1541),
}

In [30]:
def count_sequences(cfr_tree):
    all_nodes = reduce(lambda x, y: x + y.nodes, cfr_tree.information_sets.values(), [])
    all_leaves = list(filter(lambda n: n.isLeaf(), reduce(lambda x, y: x + y.children, all_nodes, [])))
    all_nodes = all_nodes + all_leaves
    
    count = 0

    for p in range(cfr_tree.numOfPlayers):
        Q_raw = list(filter(lambda q: q != {}, map(lambda n: n.base_node.getSequence(p), all_nodes)))
        Q = [{}] + [dict(t) for t in {tuple(d.items()) for d in Q_raw}]
        count += len(Q)
        
    return count

def count_infosets(cfr_tree):
    return len(cfr_tree.information_sets)

def get_size_data(par):
    if 'infoset_amount' in par:
        return (par['infoset_amount'], -1)
    
    name = parameters_to_string(par)
    if name not in size_data:
        if par['game'] == 'kuhn':
            cfr_tree = CFRTree(build_kuhn_tree(par['players'], par['rank']))
        elif par['game'] == 'leduc':
            cfr_tree = CFRTree(build_leduc_tree(par['players'], par['suits'], par['rank'], 
                                                par['betting_parameters']))
        elif par['game'] == 'goofspiel':
            # Tie solver is not needed to determine the size of the game
            cfr_tree = CFRTree(build_goofspiel_tree(par['players'], par['rank']))
        size_data[name] = (count_infosets(cfr_tree), count_sequences(cfr_tree))
        
    return size_data[name]

In [31]:
from IPython.display import HTML, display
import tabulate

In [32]:
table_text = [['Game', 'Inf.', 'Delta'] + (list(map(lambda el: str(el) + '%', target_epsilons))) + ['social welfare']]
for (game, data) in sorted(data_per_instance.items()):
    
    #if 'average' not in game:
    #    continue
    
    par = data['parameters']
    
    if 'avg_infoset_count' in data:
        size = (data['avg_infoset_count'], -1)
    size = get_size_data(par)
    
    line_text = [game, size[0]]
    line_text.append(delta_utility(par['game'], par['players'], par['rank'], par['betting_parameters']))
    #line_text += list(map(lambda t: 'n/a' if t == -1 else str(t), data['time_to_target']))
    
    for (i, t) in enumerate(data['time_to_target']):
        if t == -1:
            line_text.append('n/a')
            continue
        
        s = str(round(t, 2))
        if 'time_to_target_variance' in data:
            s += " (+- " + str(round(data['time_to_target_confidence'][i], 2)) + ")"
        line_text.append(s)
        
    if 'avg_sw' in data:
        line_text.append(str(data['avg_sw']) + "%")
    else:
        line_text.append(str(data['percent_sw']) + "%")
    
    table_text.append(line_text)
    
    print(game + " done")

goofspiel_2_4_accumulate done
goofspiel_2_4_accumulate_ done
goofspiel_2_4_accumulate__ done
goofspiel_2_4_accumulate___ done
goofspiel_2_4_accumulate____ done
goofspiel_2_4_accumulate_____ done
goofspiel_2_4_accumulate______ done
goofspiel_2_4_accumulate_______ done
goofspiel_2_4_accumulate________ done
goofspiel_2_4_accumulate_________ done
goofspiel_2_4_accumulate_average done
goofspiel_2_4_discard_if_all done
goofspiel_2_4_discard_if_all_ done
goofspiel_2_4_discard_if_all__ done
goofspiel_2_4_discard_if_all___ done
goofspiel_2_4_discard_if_all____ done
goofspiel_2_4_discard_if_all_____ done
goofspiel_2_4_discard_if_all______ done
goofspiel_2_4_discard_if_all_______ done
goofspiel_2_4_discard_if_all________ done
goofspiel_2_4_discard_if_all_________ done
goofspiel_2_4_discard_if_all_average done
goofspiel_3_4_accumulate done
goofspiel_3_4_accumulate_ done
goofspiel_3_4_accumulate__ done
goofspiel_3_4_accumulate___ done
goofspiel_3_4_accumulate____ done
goofspiel_3_4_accumulate_____ 

In [33]:
display(HTML(tabulate.tabulate(table_text, tablefmt='html')))

Game,Inf.,Delta,0.1%,0.05%,0.01%,0.005%,0.001%,0.0005%,social welfare
goofspiel_2_4_accumulate,4856,10,79.85,691.71,n/a,n/a,n/a,n/a,97.38596491228641%
goofspiel_2_4_accumulate_,4856,10,88.71,498.53,n/a,n/a,n/a,n/a,97.37294290511788%
goofspiel_2_4_accumulate__,4856,10,60.8,525.74,n/a,n/a,n/a,n/a,97.20395833333384%
goofspiel_2_4_accumulate___,4856,10,59.95,635.93,n/a,n/a,n/a,n/a,97.37926065163501%
goofspiel_2_4_accumulate____,4856,10,65.33,777.48,n/a,n/a,n/a,n/a,97.38732157850205%
goofspiel_2_4_accumulate_____,4856,10,90.97,709.52,n/a,n/a,n/a,n/a,97.50950586265557%
goofspiel_2_4_accumulate______,4856,10,62.58,592.62,n/a,n/a,n/a,n/a,97.54966914805469%
goofspiel_2_4_accumulate_______,4856,10,64.1,631.31,n/a,n/a,n/a,n/a,97.47319932999247%
goofspiel_2_4_accumulate________,4856,10,71.22,650.28,n/a,n/a,n/a,n/a,97.26794238682436%
goofspiel_2_4_accumulate_________,4856,10,72.44,602.49,n/a,n/a,n/a,n/a,97.18565989847676%


### Random

#### SCFR

In [47]:
display(HTML(tabulate.tabulate(table_text, tablefmt='html')))

Game,Inf.,Delta,0.1%,0.05%,0.01%,0.005%,0.001%,0.0005%,social welfare
random_3_12_2_average,3071,1,45.0 (+- 4.95),104.08 (+- 10.71),790.41 (+- 96.39),2138.05 (+- 394.46),36537.92 (+- 10024.92),11338.0 (+- 11514.69),90.68533764113121%
random_3_15_2_average,24542,1,605.67 (+- 74.98),1290.57 (+- 146.11),7522.02 (+- 1069.77),16128.21 (+- 2686.78),12348.02 (+- -1),28211.59 (+- -1),92.41492052055774%


#### RCFR

In [38]:
display(HTML(tabulate.tabulate(table_text, tablefmt='html')))

Game,Inf.,Delta,0.1%,0.05%,0.01%,0.005%,0.001%,0.0005%,social welfare
random_3_12_2_average,3052,1,7.67 (+- 2.53),16.94 (+- 5.83),97.13 (+- 33.43),199.93 (+- 68.45),1021.24 (+- 356.19),1446.79 (+- 582.55),89.72752028122675%
random_3_15_2_average,24588,1,190.43 (+- 107.94),214.98 (+- 107.65),541.78 (+- 344.72),893.84 (+- 599.86),4763.04 (+- 2871.42),10985.92 (+- 7169.26),93.11019014090292%


### Goofspiel 3_4 (NIPS)

#### SCFR

In [74]:
display(HTML(tabulate.tabulate(table_text, tablefmt='html')))

Game,Inf.,Delta,0.1%,0.05%,0.01%,0.005%,0.001%,0.0005%,social welfare
goofspiel_3_4_accumulate_average,98508,10,379.21 (+- 42.59),5594.12 (+- 817.15),n/a,n/a,n/a,n/a,99.57435229717778%
goofspiel_3_4_discard_always_average,98508,10,143.27 (+- 26.08),1954.18 (+- 354.93),37522.76 (+- 10272.88),55957.82 (+- 15938.51),52614.43 (+- 24261.21),63019.44 (+- 25521.5),69.23593545710295%
goofspiel_3_4_discard_if_all_average,98508,10,557.97 (+- 58.74),4438.38 (+- 794.18),61936.24 (+- -1),n/a,n/a,n/a,98.6611011166768%
goofspiel_3_4_discard_if_high_average,98508,10,324.32 (+- 49.05),2853.78 (+- 279.12),42709.87 (+- 8711.42),70809.63 (+- 19972.36),79885.63 (+- -1),n/a,88.65005547359092%


#### RCFR (temporary)

In [25]:
display(HTML(tabulate.tabulate(table_text, tablefmt='html')))

Game,Inf.,Delta,0.1%,0.05%,0.01%,0.005%,0.001%,0.0005%,social welfare
goofspiel_3_4_accumulate_average,48966,10,3621.61 (+- 5306.55),3798.92 (+- 5267.07),11007.57 (+- 17719.01),15216.13 (+- 23511.25),18263.62 (+- 31747.88),2065.72 (+- -1),99.99999999999848%
goofspiel_3_4_discard_always_average,48966,10,835.17 (+- 657.87),4903.94 (+- 7994.93),5908.29 (+- 4031.5),18002.43 (+- 4579.89),n/a,n/a,73.03009198004492%
goofspiel_3_4_discard_if_all_average,48966,10,565.9 (+- 184.9),738.69 (+- 210.12),3715.39 (+- 733.98),6625.16 (+- 2251.05),n/a,n/a,100.0%
goofspiel_3_4_discard_if_high_average,48966,10,839.58 (+- 692.41),998.0 (+- 659.98),8498.78 (+- 11262.09),14902.04 (+- 16386.67),31848.43 (+- 11462.4),55633.34 (+- 27736.58),100.0%


### Leduc (NIPS)

#### SCFR

In [78]:
display(HTML(tabulate.tabulate(table_text, tablefmt='html')))

Game,Inf.,Seq.,Delta,0.1%,0.05%,0.01%,0.005%,0.001%,0.0005%
leduc_3_3_4_average,1200,2403,21,4.0 (+- 2.3),633.25 (+- 154.35),n/a,n/a,n/a,n/a
leduc_3_3_6_average,2664,5331,21,21.54 (+- 9.72),7514.08 (+- 202.58),n/a,n/a,n/a,n/a
leduc_3_3_8_average,4704,9411,21,35.3 (+- 20.21),50103.64 (+- 5373.3),n/a,n/a,n/a,n/a


#### RCFR

In [35]:
display(HTML(tabulate.tabulate(table_text, tablefmt='html')))

Game,Inf.,Delta,0.1%,0.05%,0.01%,0.005%,0.001%,0.0005%,social welfare
leduc_3_3_4,1200,21,1.71,75.83,3717.32,22244.01,n/a,n/a,0%
leduc_3_3_6,2664,21,8.2,160.21,9314.02,40771.38,n/a,n/a,0%
leduc_3_3_8,4704,21,464.89,1222.42,63152.23,n/a,n/a,n/a,0%


### Kuhn (NIPS)

#### SCFR

In [69]:
display(HTML(tabulate.tabulate(table_text, tablefmt='html')))

Game,Inf.,Seq.,Delta,0.1%,0.05%,0.01%,0.005%,0.001%,0.0005%
kuhn_3_10_average,120,243,6,1.89 (+- 0.66),22.69 (+- 5.96),40749.64 (+- 6292.85),n/a,n/a,n/a
kuhn_3_6_average,72,147,6,0.22 (+- 0.06),1.41 (+- 0.38),1460.04 (+- 265.44),33334.91 (+- 7179.25),n/a,n/a
kuhn_3_7_average,84,171,6,0.62 (+- 0.31),4.22 (+- 1.36),3825.62 (+- 1569.9),61871.17 (+- 21964.93),n/a,n/a


#### RCFR

In [25]:
display(HTML(tabulate.tabulate(table_text, tablefmt='html')))

Game,Inf.,Delta,0.1%,0.05%,0.01%,0.005%,0.001%,0.0005%,social welfare
kuhn_3_10_average,120,6,7.21 (+- 0.19),7.21 (+- 0.19),17.2 (+- 0.45),72.78 (+- 1.56),1901.22 (+- 20.62),15086.15 (+- 187.72),-1%
kuhn_3_6_average,72,6,1.03 (+- 0.03),1.03 (+- 0.03),4.55 (+- 0.1),13.41 (+- 0.26),67.86 (+- 0.77),681.55 (+- 10.16),-1%
kuhn_3_7_average,84,6,2.35 (+- 0.34),2.35 (+- 0.34),7.92 (+- 1.25),14.33 (+- 2.4),649.72 (+- 12.36),3087.58 (+- 29.1),-1%


### Table for number of iterations instead of running time

In [24]:
table_text2 = [['Targets'] + (list(map(str, target_epsilons)))]
for (game, data) in sorted(data_per_instance.items()):
    line_text = [game] + list(map(lambda t: 'n/a' if t == -1 else str(t), data['iterations_to_target']))
    table_text2.append(line_text)

In [25]:
display(HTML(tabulate.tabulate(table_text2, tablefmt='html')))

Targets,0.1,0.05,0.01,0.005,0.001,0.0005
leduc_3_3_4,50,1500,n/a,n/a,n/a,n/a
leduc_3_3_4_,50,1400,n/a,n/a,n/a,n/a
leduc_3_3_4__,50,1450,n/a,n/a,n/a,n/a
leduc_3_3_4___,50,1400,n/a,n/a,n/a,n/a
leduc_3_3_4____,50,1450,n/a,n/a,n/a,n/a
leduc_3_3_4_____,100,1450,n/a,n/a,n/a,n/a
leduc_3_3_4______,50,1400,n/a,n/a,n/a,n/a
leduc_3_3_4_______,150,1800,n/a,n/a,n/a,n/a
leduc_3_3_4________,150,1700,n/a,n/a,n/a,n/a
leduc_3_3_4_________,100,1300,n/a,n/a,n/a,n/a


## Time comparisons

In [ ]:
def compare(files, xlims_time = None, xlims_iter = None, ylims = None, legend = None, title = 'Comparison'):
    fig_time, ax_time = plt.subplots()
    fig_iter, ax_iter = plt.subplots()
    
    for filename in files:
        file = open(filename, "r")
        data = json.load(file)
        file.close()

        print("File = " + filename)
        print("Game = " + parameters_to_string(data['parameters']))

        print("Total iterations = " + str(data['parameters']['number_iterations']))
        print("Check every iteration = " + str(data['parameters']['check_every_iteration']))

        durations = list(map(lambda el: el['duration'], data['data']))

        du = delta_utility(data['parameters']['game'], data['parameters']['players'], data['parameters']['rank'],
                   data['parameters']['betting_parameters'], data['parameters']['color_distribution'], 
                   data['parameters']['suits'])

        iteration_counts = list(map(lambda el: el['iteration_number'], data['data']))
        epsilons = list(map(lambda el: max(0, -min(el['epsilon'])), data['data']))
        percent_epsilons = list(map(lambda el: el / du, epsilons))
        cum_durations = [sum(durations[:i]) for i in range(len(durations))]
        
        ax_time.plot(cum_durations, percent_epsilons)
        ax_iter.plot(iteration_counts, percent_epsilons)
        
    if legend == None:
        legend = ["Result " + str(i+1) for i in range(len(files))]
    ax_time.legend(legend)
    ax_iter.legend(legend)
    ax_time.set(xlabel = "Time [s]", ylabel = "Epsilon [%]")
    ax_iter.set(xlabel = "Iterations", ylabel = "Epsilon [%]")
    if ylims != None:
        ax_time.set_ylim(ylims)
        ax_iter.set_ylim(ylims)
    if xlims_time != None:
        ax_time.set_xlim(xlims_time)
    if xlims_iter != None:
        ax_iter.set_xlim(xlims_iter)
    ax_time.set_title(title)
    ax_iter.set_title(title)

In [ ]:
compare(['results/remote/RCFR/kuhn/1553957872_3_8', 'results/remote/kuhn_20190120/1547765102_3_8'],
        xlims = (0, 15000), ylims = (0, 0.02), legend = ['RCFR', 'SCFR'], title = 'Kuhn 3_8')
compare(['results/remote/RCFR/kuhn/1553957872_3_8', 'results/remote/kuhn_short_20190121/1548008650_3_8'],
        xlims = (0, 15000), ylims = (0, 0.02), legend = ['RCFR', 'SCFR'], title = 'Kuhn 3_8')
compare(['results/remote/RCFR/kuhn/1553957868_3_7', 'results/remote/kuhn_20190120/1547765096_3_7'],
        xlims = (0, 7000), ylims = (0, 0.02), legend = ['RCFR', 'SCFR'], title = 'Kuhn 3_7')

In [ ]:
compare(['results/remote/RCFR/kuhn/1553957883_3_10', 'results/remote/RCFR_notopt/kuhn/1554108461_3_10'],
        xlims = (0, 10000), legend = ['RCFR', 'RCFR_nopt'], title = 'Kuhn 3_10')

In [ ]:
compare(['results/remote/RCFR/kuhn/1553957872_3_8', 'results/remote/RCFR_notopt/kuhn/1554108461_3_8',
         'results/remote/kuhn_20190120/1547765102_3_8'],
        xlims = (0, 5000), legend = ['RCFR', 'RCFR_nopt', 'SCFR'], title = 'Kuhn 3_8')
compare(['results/remote/RCFR/kuhn/1553957872_3_8', 'results/remote/RCFR_notopt/kuhn/1554108461_3_8',
         'results/remote/kuhn_20190120/1547765102_3_8'],
        xlims = (0, 1000), ylims = (0, 0.05), legend = ['RCFR', 'RCFR_nopt', 'SCFR'], title = 'Kuhn 3_8')

In [ ]:
compare(['results/remote/RCFR_notopt/goofspiel/1554199851_3_4', 
         'results/remote/goofspiel_long_20180122/1548105218_3_4'], legend = ["RCFR_nopt", "SCFR"],
         title = "Goofspiel 3_4")

compare(['results/remote/RCFR_notopt/goofspiel/1554382206_3_4', 
         'results/remote/goofspiel_long_20180122/1548105218_3_4'], legend = ["RCFR_nopt", "SCFR"],
         title = "Goofspiel 3_4")

In [ ]:
compare(['results/remote/RCFR_notopt/hanabi/1554413019_2_2_[2_1]_1_1', 
         'results/remote/SCFR_hanabi/hanabi/1554461737_2_2_[2_1]_1_1'], legend = ["RCFR_nopt", "SCFR"],
         title = "Hanabi 2 colors [2, 1] distribution")

In [ ]:
compare(['results/remote/hanabi_tests/RCFR_nopt/hanabi/1554739324_3_1_[2_2_1]_1_1',
         'results/remote/hanabi_tests/SCFR/hanabi/1554739324_3_1_[2_2_1]_1_1'], ylims = (0, 0.02), 
        legend = ["RCFR", "SCFR"])

In [ ]:
compare(['results/remote/hanabi_tests/RCFR/hanabi/1554830690_2_1_[2_2_1]_1_1',
         'results/remote/hanabi_tests/RCFR_nopt/hanabi/1554830690_2_1_[2_2_1]_1_1',
         'results/remote/hanabi_tests/SCFR/hanabi/1554830690_2_1_[2_2_1]_1_1'], 
         legend = ["RCFR", "RCFR_nopt", "SCFR"], xlims = (0, 10000), ylims = (0, 0.05))

In [ ]:
compare(['results/remote/hanabi_tests/SCFR/hanabi/1554741241_3_2_[2_1]_1_1',
         'results/remote/hanabi_tests/RCFR_nopt/hanabi/1554741154_3_2_[2_1]_1_1'], 
         legend = ["SCFR", "RCFR_nopt"])

In [ ]:
compare(['results/remote/hanabi_tests/RCFR_nopt/hanabi/1554831012_2_2_[2_1]_1_1',
         'results/remote/hanabi_tests/SCFR/hanabi/1554831004_2_2_[2_1]_1_1'], ylims = (0, 0.2), 
        legend = ["RCFR_nopt", "SCFR"])

---

## SCFR/CFR/CFR+ comparisons

In [ ]:
import numpy as np

In [ ]:
def compare_algorithms(filename, filename_cfr, filename_cfr_plus, cumsum_durations = False):
    file = open(filename, "r")
    data = json.load(file)
    file.close()

    file_cfr = open(filename_cfr, "r")
    data_cfr = json.load(file_cfr)
    file_cfr.close()

    file_cfr_plus = open(filename_cfr_plus, "r")
    data_cfr_plus = json.load(file_cfr_plus)
    file_cfr_plus.close()    

    durations = list(map(lambda el: el['duration'], data['data']))
    durations_cfr = list(map(lambda el: el['duration'], data_cfr['data']))
    durations_cfr_plus = list(map(lambda el: el['duration'], data_cfr_plus['data']))

    du = delta_utility(data['parameters']['game'], data['parameters']['players'], data['parameters']['rank'],
                   data['parameters']['betting_parameters'])
    du_cfr = delta_utility(data_cfr['parameters']['game'], data_cfr['parameters']['players'], 
                           data_cfr['parameters']['rank'], data_cfr['parameters']['betting_parameters'])
    du_cfr_plus = delta_utility(data_cfr_plus['parameters']['game'], data_cfr_plus['parameters']['players'], 
                           data_cfr_plus['parameters']['rank'], data_cfr_plus['parameters']['betting_parameters'])

    iteration_counts = list(map(lambda el: el['iteration_number'], data['data']))
    epsilons = list(map(lambda el: max(0, -min(el['epsilon'])), data['data']))
    percent_epsilons = list(map(lambda el: el / du, epsilons))
    iteration_counts_cfr = list(map(lambda el: el['iteration_number'], data_cfr['data']))
    epsilons_cfr = list(map(lambda el: max(0, -min(el['epsilon'])), data_cfr['data']))
    percent_epsilons_cfr = list(map(lambda el: el / du_cfr, epsilons_cfr))
    iteration_counts_cfr_plus = list(map(lambda el: el['iteration_number'], data_cfr_plus['data']))
    epsilons_cfr_plus = list(map(lambda el: max(0, -min(el['epsilon'])), data_cfr_plus['data']))
    percent_epsilons_cfr_plus = list(map(lambda el: el / du_cfr_plus, epsilons_cfr_plus))
    utilities = list(map(lambda el: el['utility'], data['data']))
    utilities_cfr = list(map(lambda el: el['utility'], data_cfr['data']))
    utilities_cfr_plus = list(map(lambda el: el['utility'], data_cfr_plus['data']))

    print(data['parameters'])
    if 'total_duration' in data:
        print("Duration = " + str(data['total_duration']) + " seconds")
        print("Utility = " + str(data['utility']))
        print("Social welfare = " + str(sum(data['utility'])))
        print("Max delta utility = " + str(max(data['utility']) - min(data['utility'])))
    else:
        print("--- Computation was stopped before the end ---")
        print("Duration = " + str(sum(durations)) + " seconds")
        print("Last logged utility = " + str(utilities[len(utilities)-1]))
        print("Social welfare = " + str(sum(utilities[len(utilities)-1])))
        print("Max delta utility = " + str(max(utilities[len(utilities)-1]) - min(utilities[len(utilities)-1])))
    print('')
    print(data_cfr['parameters'])
    if 'total_duration' in data_cfr:
        print("Duration = " + str(data_cfr['total_duration']) + " seconds")
        print("Utility = " + str(data_cfr['utility']))
        print("Social welfare = " + str(sum(data_cfr['utility'])))
        print("Max delta utility = " + str(max(data_cfr['utility']) - min(data_cfr['utility'])))
    else:
        print("--- Computation was stopped before the end ---")
        print("Duration = " + str(sum(durations_cfr)) + " seconds")
        print("Last logged utility = " + str(utilities_cfr[len(utilities_cfr)-1]))
        print("Social welfare = " + str(sum(utilities_cfr[len(utilities_cfr)-1])))
        print("Max delta utility = " + str(max(utilities_cfr[len(utilities_cfr)-1]) - \
                                           min(utilities_cfr[len(utilities_cfr)-1])))
    print('')
    print(data_cfr_plus['parameters'])
    if 'total_duration' in data_cfr_plus:
        print("Duration = " + str(data_cfr_plus['total_duration']) + " seconds")
        print("Utility = " + str(data_cfr_plus['utility']))
        print("Social welfare = " + str(sum(data_cfr_plus['utility'])))
        print("Max delta utility = " + str(max(data_cfr_plus['utility']) - min(data_cfr_plus['utility'])))
    else:
        print("--- Computation was stopped before the end ---")
        print("Duration = " + str(sum(durations_cfr_plus)) + " seconds")
        print("Last logged utility = " + str(utilities_cfr_plus[len(utilities_cfr_plus)-1]))
        print("Social welfare = " + str(sum(utilities_cfr_plus[len(utilities_cfr_plus)-1])))
        print("Max delta utility = " + str(max(utilities_cfr_plus[len(utilities_cfr_plus)-1]) - \
                                           min(utilities_cfr_plus[len(utilities_cfr_plus)-1])))
    print('')

    #plt.plot(iteration_counts, epsilons)
    #plt.plot(iteration_counts_cfr, epsilons_cfr)
    #plt.plot(iteration_counts_cfr_plus, epsilons_cfr_plus)
    #plt.show()
    
    plt.plot(iteration_counts, percent_epsilons)
    plt.plot(iteration_counts_cfr, percent_epsilons_cfr)
    plt.plot(iteration_counts_cfr_plus, percent_epsilons_cfr_plus)
    plt.legend(['scfr','cfr', 'cfr+'])
    plt.show()
    
    plt.plot(np.cumsum(durations), percent_epsilons)
    plt.plot(np.cumsum(durations_cfr) if cumsum_durations else durations_cfr, percent_epsilons_cfr)
    plt.plot(np.cumsum(durations_cfr_plus) if cumsum_durations else durations_cfr_plus, percent_epsilons_cfr_plus)
    plt.legend(['scfr','cfr', 'cfr+'])
    plt.show()

In [ ]:
compare_algorithms('results/remote/goofspiel_cfr_comparisons_20190122/goofspiel_3_4_scfr_accumulate',
                   'results/remote/goofspiel_cfr_comparisons_20190122/goofspiel_3_4_cfr_accumulate',
                   'results/remote/goofspiel_cfr_comparisons_20190122/goofspiel_3_4_cfr_plus_accumulate')

In [ ]:
compare_algorithms('results/remote/goofspiel_cfr_comparisons_20190122/goofspiel_3_3_scfr_accumulate',
                   'results/remote/goofspiel_cfr_comparisons_20190122/goofspiel_3_3_cfr_accumulate',
                   'results/remote/goofspiel_cfr_comparisons_20190122/goofspiel_3_3_cfr_plus_accumulate',
                   cumsum_durations = True)

In [ ]:
compare_algorithms('results/remote/goofspiel_cfr_comparisons_20190122/goofspiel_3_3_scfr_discard_if_all',
                   'results/remote/goofspiel_cfr_comparisons_20190122/goofspiel_3_3_cfr_discard_if_all', 
                   'results/remote/goofspiel_cfr_comparisons_20190122/goofspiel_3_3_cfr_plus_discard_if_all',
                   cumsum_durations = True)

In [ ]:
compare_algorithms('results/remote/goofspiel_cfr_comparisons_20190122/goofspiel_3_3_scfr_discard_if_high',
                   'results/remote/goofspiel_cfr_comparisons_20190122/goofspiel_3_3_cfr_discard_if_high',
                   'results/remote/goofspiel_cfr_comparisons_20190122/goofspiel_3_3_cfr_plus_discard_if_high',
                   cumsum_durations = True)

In [ ]:
compare_algorithms('results/remote/goofspiel_cfr_comparisons_20190122/goofspiel_3_3_scfr_discard_always',
                   'results/remote/goofspiel_cfr_comparisons_20190122/goofspiel_3_3_cfr_discard_always',
                   'results/remote/goofspiel_cfr_comparisons_20190122/goofspiel_3_3_cfr_plus_discard_always',
                   cumsum_durations = True)

In [ ]:
compare_algorithms('results/remote/goofspiel_long_20180122/1548105233_3_4',
                   'results/remote/goofspiel_cfr_comparisons_20190122/goofspiel_3_4_cfr_discard_always_ALBO',
                   'results/remote/goofspiel_cfr_comparisons_20190122/goofspiel_3_4_cfr_plus_discard_always_ALBO',
                   cumsum_durations = True)

In [ ]:
compare_algorithms('results/remote/goofspiel_long_20180122/1548105230_3_4',
                   'results/remote/goofspiel_cfr_comparisons_20190122/goofspiel_3_4_cfr_discard_if_high_ALBO',
                   'results/remote/goofspiel_cfr_comparisons_20190122/goofspiel_3_4_cfr_plus_discard_if_high_ALBO',
                   cumsum_durations = True)

In [ ]:
compare_algorithms('results/remote/goofspiel_long_20180122/1548105224_3_4',
                   'results/remote/goofspiel_cfr_comparisons_20190122/goofspiel_3_4_cfr_discard_if_all_ALBO',
                   'results/remote/goofspiel_cfr_comparisons_20190122/goofspiel_3_4_cfr_plus_discard_if_all_ALBO',
                   cumsum_durations = True)

## Data to CSV for tikz

In [ ]:
f1 = open('results/remote/goofspiel_long_20180122/1548105233_3_4', 'r')
f2 = open('results/remote/goofspiel_cfr_comparisons_20190122/goofspiel_3_4_cfr_discard_always_ALBO', 'r')
f3 = open('results/remote/goofspiel_cfr_comparisons_20190122/goofspiel_3_4_cfr_plus_discard_always_ALBO', 'r')

f4 = open('results/remote/goofspiel_cfr_comparisons_20190122/goofspiel_3_4_scfr_accumulate', 'r')
f5 = open('results/remote/goofspiel_cfr_comparisons_20190122/goofspiel_3_4_cfr_accumulate', 'r')
f6 = open('results/remote/goofspiel_cfr_comparisons_20190122/goofspiel_3_4_cfr_plus_accumulate', 'r')

data1 = json.load(f1)
data2 = json.load(f2)
data3 = json.load(f3)

data4 = json.load(f4)
data5 = json.load(f5)
data6 = json.load(f6)

durations1 = list(map(lambda el: el['duration'], data1['data']))
durations2 = list(map(lambda el: el['duration'], data2['data']))
durations3 = list(map(lambda el: el['duration'], data3['data']))

durations4 = list(map(lambda el: el['duration'], data4['data']))
durations5 = list(map(lambda el: el['duration'], data5['data']))
durations6 = list(map(lambda el: el['duration'], data6['data']))

du = delta_utility('goofspiel', 3, 4)

with open("cfr_comparison_3_4_discard_always_scfr.csv", 'w') as o:
    o.write('iteration, epsilon, t\n')
    
    for i in range(len(data1['data'])):
        d = data1['data'][i]
        o.write(str(d['iteration_number']) + ', ' + str(-min(d['epsilon']) / du) + ', ' + \
                str(sum(durations1[:i+1])) + "\n")
        
with open("cfr_comparison_3_4_discard_always_cfr.csv", 'w') as o:
    o.write('iteration, epsilon, t\n')
    
    for i in range(len(data2['data'])):
        d = data2['data'][i]
        o.write(str(d['iteration_number']) + ', ' + str(-min(d['epsilon']) / du) + ', ' + \
                str(sum(durations2[:i+1])) + "\n")
        
with open("cfr_comparison_3_4_discard_always_cfr_plus.csv", 'w') as o:
    o.write('iteration, epsilon, t\n')
    
    for i in range(len(data3['data'])):
        d = data3['data'][i]
        o.write(str(d['iteration_number']) + ', ' + str(-min(d['epsilon']) / du) + ', ' + \
                str(sum(durations3[:i+1])) + "\n")

with open("cfr_comparison_3_4_accumulate.csv", 'w') as o:
    o.write('iteration, scfr, cfr, cfr+\n')
    
    iterations = list(map(lambda el: el['iteration_number'], data4['data']))
    epsilons4 = list(map(lambda el: -min(el['epsilon']) / du, data4['data']))
    epsilons5 = list(map(lambda el: -min(el['epsilon']) / du, data5['data']))
    epsilons6 = list(map(lambda el: -min(el['epsilon']) / du, data6['data']))
    
    for (it, s, c, cp) in zip(iterations, epsilons4, epsilons5, epsilons6):
        o.write(str(it) + ', ' + str(s) + ', ' + str(c) + ', ' + str(cp) + '\n')
        
f1.close()
f2.close()
f3.close()

f4.close()
f5.close()
f6.close()

In [ ]:
icml_data_folder  = "../../../../Dropbox/Honours Programme/Shared Material/ICML-19/imgs/data/"

In [ ]:
def print_csv(name1, name2, name3):
    f1 = open(name1, 'r')
    f2 = open(name2, 'r')
    f3 = open(name3, 'r')

    data1 = json.load(f1)
    data2 = json.load(f2)
    data3 = json.load(f3)

    durations1 = list(map(lambda el: el['duration'], data1['data']))
    durations2 = list(map(lambda el: el['duration'], data2['data']))
    durations3 = list(map(lambda el: el['duration'], data3['data']))

    du = delta_utility('goofspiel', 3, 4)

    shortest_run = min(sum(durations1), sum(durations2), sum(durations3))
    
    ts = data1['parameters']['tie_solver']

    with open(icml_data_folder + "cfr_comparison_3_4_" + ts + "_scfr.csv", 'w') as o:
        o.write('iteration, epsilon, t\n')

        for i in range(len(data1['data'])):
            if sum(durations1[:i+1]) > shortest_run:
                break

            d = data1['data'][i]
            o.write(str(d['iteration_number']) + ', ' + str(-min(d['epsilon'] + [0]) / du) + ', ' + \
                    str(sum(durations1[:i+1])) + "\n")

    with open(icml_data_folder + "cfr_comparison_3_4_" + ts + "_cfr.csv", 'w') as o:
        o.write('iteration, epsilon, t\n')

        for i in range(len(data2['data'])):
            if sum(durations2[:i+1]) > shortest_run:
                break

            d = data2['data'][i]
            o.write(str(d['iteration_number']) + ', ' + str(-min(d['epsilon'] + [0]) / du) + ', ' + \
                    str(sum(durations2[:i+1])) + "\n")

    with open(icml_data_folder + "cfr_comparison_3_4_" + ts + "_cfr_plus.csv", 'w') as o:
        o.write('iteration, epsilon, t\n')

        for i in range(len(data3['data'])):
            if sum(durations3[:i+1]) > shortest_run:
                break

            d = data3['data'][i]
            o.write(str(d['iteration_number']) + ', ' + str(-min(d['epsilon'] + [0]) / du) + ', ' + \
                    str(sum(durations3[:i+1])) + "\n")

    f1.close()
    f2.close()
    f3.close()

In [ ]:
print_csv('results/remote/goofspiel_long_20180122/1548105233_3_4',
          'results/remote/goofspiel_cfr_comparisons_20190122/goofspiel_3_4_cfr_discard_always_ALBO',
          'results/remote/goofspiel_cfr_comparisons_20190122/goofspiel_3_4_cfr_plus_discard_always_ALBO')

In [ ]:
print_csv('results/remote/goofspiel_long_20180122/1548105230_3_4',
                   'results/remote/goofspiel_cfr_comparisons_20190122/goofspiel_3_4_cfr_discard_if_high_ALBO',
                   'results/remote/goofspiel_cfr_comparisons_20190122/goofspiel_3_4_cfr_plus_discard_if_high_ALBO')

In [ ]:
print_csv('results/remote/goofspiel_long_20180122/1548105224_3_4',
                   'results/remote/goofspiel_cfr_comparisons_20190122/goofspiel_3_4_cfr_discard_if_all_ALBO',
                   'results/remote/goofspiel_cfr_comparisons_20190122/goofspiel_3_4_cfr_plus_discard_if_all_ALBO')